# Two-Opt Route Optimization with Numba

This notebook demonstrates the performance improvement from using Numba's JIT compilation on the two-opt algorithm for optimizing routes through world capitals.

In [ ]:
import sys
sys.path.append('../src')

import numpy as np
import numba
from get_input import _get_all_capital_geocodes, get_distance_matrix
from two_opt import save_christmas
from helper import get_distance
from visualization import get_map

## Load Data

In [2]:
locations = _get_all_capital_geocodes()
distance_matrix = get_distance_matrix()
initial_route = np.arange(len(locations))

## Run Python Two-Opt

In [ ]:

two_opt_route = save_christmas(initial_route, distance_matrix)
distance = get_distance(two_opt_route, distance_matrix)

## Jit the Function with Numba ⚡

In [ ]:
save_christmas_fast = numba.njit(save_christmas)
two_opt_route_numba = save_christmas_fast(initial_route, distance_matrix)

## Results are the same

In [5]:
assert (two_opt_route_numba == two_opt_route).all()

### Let us benmark their speed

#### Vanilla Python:

In [ ]:
%%timeit
np.random.shuffle(initial_route)
save_christmas(initial_route, distance_matrix)

113 ms ± 2.75 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### Numba:

In [ ]:
%%timeit
np.random.shuffle(initial_route)
save_christmas_fast(initial_route, distance_matrix)

751 μs ± 8.45 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Numba made this function ~150x faster!

#### Faster computation means more time for finding a shorter route!

In [8]:
best_route = two_opt_route_numba
best_distance = get_distance(best_route, distance_matrix)

In [ ]:
for i in range(150):   
    np.random.shuffle(initial_route)
    new_route = save_christmas_fast(initial_route, distance_matrix)
    new_distance = get_distance(new_route, distance_matrix)
    if new_distance < best_distance:
        print(f"Found new route that is {best_distance-new_distance:.2f} km shorter!")
        best_distance = new_distance
        best_route = new_route

Found new route that is 4382.49 km shorter!
Found new route that is 618.00 km shorter!
Found new route that is 2033.75 km shorter!
Found new route that is 7864.36 km shorter!


In [10]:
print(f"New route for Santa is {(distance-best_distance)/distance * 100:.2f}% shorter, thanks to Numba🐍⚡")

New route for Santa is 7.61% shorter, thanks to Numba🐍⚡


In [11]:
get_map(best_route, locations)